In [1]:
import requests
url = "https://www.infocasas.com.pe/vendo-hermosa-casa-de-3-pisos-comas/192540025"
headers = {"User-Agent": "Mozilla/5.0"}
html = requests.get(url, headers=headers)
html

<Response [200]>

## 1 ejemplo

In [ ]:
import requests
import json
from bs4 import BeautifulSoup
import re

url = "https://www.infocasas.com.pe/vendo-hermosa-casa-de-3-pisos-comas/192540025"
headers = {"User-Agent": "Mozilla/5.0"}

# Obtener HTML
res = requests.get(url, headers=headers)
soup = BeautifulSoup(res.text, "html.parser")

# Buscar el bloque de JavaScript que contiene los datos
script_tags = soup.find_all("script", text=re.compile("TechnicalSheet"))

# Buscar el primer bloque que contiene "TechnicalSheet"
for tag in script_tags:
    if "TechnicalSheet" in tag.text:
        raw_json = tag.string or tag.text
        break

# Extraer los objetos JSON que contienen los datos técnicos
datos = {}
matches = re.findall(r'{"__typename":"TechnicalSheet".*?}', raw_json)

for m in matches:
    item = json.loads(m)
    datos[item["text"]] = item["value"]

# Campos deseados
campos = [
    "Zona", "M² edificados", "M² del terreno", "Dormitorios", "Plantas",
    "Tipo de Propiedad", "Estado", "Baños", "Año de Construcción","Garajes"
]

# Extraer precio de venta
precio_tag = soup.select_one("span.price strong")
precio = precio_tag.get_text(strip=True) if precio_tag else "No encontrado"

# Mostrar resultado final
print("Precio de venta:", precio)
for campo in campos:
    print(f"{campo}: {datos.get(campo, 'No encontrado')}")


Precio de venta: U$S 150.000
Zona: Comas
M² edificados: 124 m2
M² del terreno: 250 m2
Dormitorios: 4
Plantas: 3
Tipo de Propiedad: Casa
Estado: Buen estado
Baños: 3
Año de Construcción: 2012


C:\Users\arthu\AppData\Local\Temp\ipykernel_2368\1682716206.py:14: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  script_tags = soup.find_all("script", text=re.compile("TechnicalSheet"))


## Obtenendo los links

In [14]:
import requests
from bs4 import BeautifulSoup
import re
import time

headers = {"User-Agent": "Mozilla/5.0"}

# 1. URL base de propiedades en Lima
url_listado = "https://www.infocasas.com.pe/venta/casas/lima"
res = requests.get(url_listado, headers=headers)
soup = BeautifulSoup(res.text, "html.parser")

# 2. Extraer enlaces únicos de cada propiedad (máx. 10 como ejemplo)
cards = soup.select("a.property-card")
urls = ["https://www.infocasas.com.pe/venta/casas/lima" + a["href"] for a in cards if "href" in a.attrs]
urls = list(set(urls))[:10]  # Evita duplicados y limita a 10

# 3. Función para scrapear cada propiedad
def scrap_propiedad(url):
    try:
        r = requests.get(url, headers=headers)
        s = BeautifulSoup(r.text, "html.parser")

        # Extraer precio
        precio_tag = s.select_one("span.price strong")
        precio = precio_tag.get_text(strip=True) if precio_tag else "No encontrado"

        # Buscar bloque JSON con ficha técnica
        script_tags = s.find_all("script", text=re.compile("TechnicalSheet"))
        raw_json = next((tag.string or tag.text for tag in script_tags if "TechnicalSheet" in tag.text), "")
        matches = re.findall(r'{"__typename":"TechnicalSheet".*?}', raw_json)

        data = {}
        for m in matches:
            try:
                item = json.loads(m)
                data[item["text"]] = item["value"]
            except:
                pass

        # Campos de interés
        campos = [
            "Zona", "M² edificados", "M² del terreno", "Dormitorios", "Plantas",
            "Tipo de Propiedad", "Estado", "Baños", "Año de Construcción"
        ]

        salida = {"URL": url, "Precio": precio}
        for campo in campos:
            salida[campo] = data.get(campo, "No encontrado")

        return salida

    except Exception as e:
        return {"URL": url, "Error": str(e)}

# 4. Recorrer todas las URLs y scrapear
resultados = []
for i, link in enumerate(urls, 1):
    print(f"[{i}/{len(urls)}] Scrapeando: {link}")
    resultados.append(scrap_propiedad(link))
    time.sleep(1)  # pausa para no saturar el servidor

# 5. Mostrar resultados
import pandas as pd
df = pd.DataFrame(resultados)

## 1er resultado

In [16]:
print("Total 'a' con href encontrados:")
for a in soup.find_all("a", href=True):
    if "/vendo" in a["href"]:
        print(a["href"])

Total 'a' con href encontrados:
/vendo-hermosa-casa-de-3-pisos-comas/192540025
/vendo-hermosa-casa-de-3-pisos-comas/192540025
/vendo-casa-de-playa-amoblada-asia-del-mar-sarapampa-ac130mt/191221321
/vendo-casa-de-playa-amoblada-asia-del-mar-sarapampa-ac130mt/191221321
/vendo-estupenda-casa-primer-piso-con-frente-avenida/190925992
/vendo-estupenda-casa-primer-piso-con-frente-avenida/190925992


## 2do resultado

In [17]:
from bs4 import BeautifulSoup
import requests

url_listado = "https://www.infocasas.com.pe/venta/casas/lima"
headers = {"User-Agent": "Mozilla/5.0"}

res = requests.get(url_listado, headers=headers)
soup = BeautifulSoup(res.text, "html.parser")

# Extraer todos los enlaces que comienzan con "/vendo-"
links = soup.find_all("a", href=True)
urls = set()  # usar set para evitar duplicados

for link in links:
    href = link["href"]
    if href.startswith("/vendo-") and href.count("/") == 2:  # filtrar solo los finales tipo /vendo/.../id
        urls.add("https://www.infocasas.com.pe" + href)

urls = list(urls)[:10]  # limita a 10 si deseas

# Verifica qué encontraste
print(f"Se encontraron {len(urls)} propiedades:")
for url in urls:
    print(url)


Se encontraron 3 propiedades:
https://www.infocasas.com.pe/vendo-estupenda-casa-primer-piso-con-frente-avenida/190925992
https://www.infocasas.com.pe/vendo-casa-de-playa-amoblada-asia-del-mar-sarapampa-ac130mt/191221321
https://www.infocasas.com.pe/vendo-hermosa-casa-de-3-pisos-comas/192540025


## Codigo webscrappiniig

In [16]:
print(df_reemplazado.isna().sum())

URL                    0
Precio                 0
Referencia             0
Zona                   0
Disposición            0
M² edificados          0
M² del terreno         0
Dormitorios            0
Plantas                0
Barrio Privado         3
Acepta permuta         3
Tipo de Propiedad      0
Estado                 0
Baños                  0
M² de terraza          3
Año de Construcción    1
Garajes                2
Sobre                  0
Cuota inicial          3
Vivienda Social        3
dtype: int64


## Codigo webscrappinig real

In [26]:
import requests
from bs4 import BeautifulSoup
import json
import re
import pandas as pd
import time

headers = {"User-Agent": "Mozilla/5.0"}

def obtener_urls_pagina(n_pagina):
    url = f"https://www.infocasas.com.pe/venta/casas/lima/pagina-{n_pagina}" if n_pagina > 1 else "https://www.infocasas.com.pe/venta/casas/lima"
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")

    links = soup.find_all("a", href=True)
    urls = set()
    for link in links:
        href = link["href"]
        if href.startswith("/vendo-") and href.count("/") == 2:
            urls.add("https://www.infocasas.com.pe" + href)
    return list(urls)

def scrap_propiedad(url):
    try:
        r = requests.get(url, headers=headers)
        s = BeautifulSoup(r.text, "html.parser")

        precio_tag = s.select_one("span.price strong")
        precio = precio_tag.get_text(strip=True) if precio_tag else "No encontrado"

        script_tags = s.find_all("script", text=re.compile("TechnicalSheet"))
        raw_json = next((tag.string or tag.text for tag in script_tags if "TechnicalSheet" in tag.text), "")
        matches = re.findall(r'{"__typename":"TechnicalSheet".*?}', raw_json)

        data = {}
        for m in matches:
            item = json.loads(m)
            data[item["text"]] = item["value"]

        campos = [
            "Referencia","Zona","Disposición", "M² edificados", "M² del terreno",
             "Dormitorios", "Plantas","Barrio Privado","Acepta permuta",
            "Tipo de Propiedad", "Estado", "Baños","M² de terraza",
              "Año de Construcción","Garajes","Sobre","Cuota inicial","Vivienda Social"
        ]

        salida = {"URL": url, "Precio": precio}
        for campo in campos:
            salida[campo] = data.get(campo, "No encontrado")

        return salida

    except Exception as e:
        return {"URL": url, "Error": str(e)}



In [ ]:
# 🔁 Recorremos varias páginas
resultados = []
pagina = 1
paginas_vacias_consecutivas = 0
LIMITE_VACIAS = 10
LIMITE_TOTAL_PAGINAS = 100  # o más si deseas

try:
    while pagina <= LIMITE_TOTAL_PAGINAS:
        print(f"\n📄 Procesando página {pagina}")
        urls = obtener_urls_pagina(pagina)

        if not urls:
            paginas_vacias_consecutivas += 1
            print(f"⚠️ Página vacía #{paginas_vacias_consecutivas}")
            if paginas_vacias_consecutivas >= LIMITE_VACIAS:
                print(f"🛑 {LIMITE_VACIAS} páginas vacías consecutivas. Se detiene.")
                break
        else:
            paginas_vacias_consecutivas = 0
            for i, link in enumerate(urls, 1):
                print(f"  [{i}/{len(urls)}] {link}")
                resultados.append(scrap_propiedad(link))
                time.sleep(1)

        pagina += 1

except KeyboardInterrupt:
    print("🛑 Interrupción manual detectada. Guardando progreso...")

finally:
    df = pd.DataFrame(resultados)
    df.to_csv("viviendas_lima_paginas_casas.csv", index=False, encoding="utf-8-sig")
    print(f"✅ Se guardaron {len(resultados)} registros en 'viviendas_lima_paginas_casas.csv'")



📄 Procesando página 1
  [1/3] https://www.infocasas.com.pe/vendo-casa-de-playa-amoblada-asia-del-mar-sarapampa-ac130mt/191221321


C:\Users\arthu\AppData\Local\Temp\ipykernel_34724\724842558.py:31: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  script_tags = s.find_all("script", text=re.compile("TechnicalSheet"))


  [2/3] https://www.infocasas.com.pe/vendo-hermosa-casa-de-3-pisos-comas/192540025
  [3/3] https://www.infocasas.com.pe/vendo-estupenda-casa-primer-piso-con-frente-avenida/190925992

📄 Procesando página 2
  [1/3] https://www.infocasas.com.pe/vendo-casa-en-la-molina-alameda-del-corregidor/192008480
  [2/3] https://www.infocasas.com.pe/vendo-casa-292-m2-en-san-borja-norte/191936403
  [3/3] https://www.infocasas.com.pe/vendo-casa-por-remodelar-marsano-con-ayacucho/192439304

📄 Procesando página 3
  [1/1] https://www.infocasas.com.pe/vendo-casa-160-m2-6-dormitorios-san-miguel/192567144

📄 Procesando página 4
⚠️ Página vacía #1

📄 Procesando página 5
  [1/1] https://www.infocasas.com.pe/vendo-casa-ubicada-frente-a-parque-en-los-olivos/192231470

📄 Procesando página 6
  [1/3] https://www.infocasas.com.pe/vendo-casa-de-3-pisos-azotea-10dorm-urb-las-malvas-callao/192560538
  [2/3] https://www.infocasas.com.pe/vendo-hermosa-casa-con-piscina-en-san-isidro-at/185986355
  [3/3] https://www.infocas

In [30]:
import requests
from bs4 import BeautifulSoup
import json
import re
import pandas as pd
import time

headers = {"User-Agent": "Mozilla/5.0"}

def obtener_urls_pagina(n_pagina):
    url = f"https://www.infocasas.com.pe/venta/departamentos/lima/pagina-{n_pagina}" if n_pagina > 1 else "https://www.infocasas.com.pe/venta/departamentos/lima"
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")

    links = soup.find_all("a", href=True)
    urls = set()
    for link in links:
        href = link["href"]
        if href.startswith("/vendo-") and href.count("/") == 2:
            urls.add("https://www.infocasas.com.pe" + href)
    return list(urls)

def scrap_propiedad(url):
    try:
        r = requests.get(url, headers=headers)
        s = BeautifulSoup(r.text, "html.parser")

        precio_tag = s.select_one("span.price strong")
        precio = precio_tag.get_text(strip=True) if precio_tag else "No encontrado"

        script_tags = s.find_all("script", text=re.compile("TechnicalSheet"))
        raw_json = next((tag.string or tag.text for tag in script_tags if "TechnicalSheet" in tag.text), "")
        matches = re.findall(r'{"__typename":"TechnicalSheet".*?}', raw_json)

        data = {}
        for m in matches:
            item = json.loads(m)
            data[item["text"]] = item["value"]

        campos = [
            "Referencia","Zona","Disposición", "M² edificados", "M² del terreno",
             "Dormitorios", "Plantas","Barrio Privado","Acepta permuta",
            "Tipo de Propiedad", "Estado", "Baños","M² de terraza",
              "Año de Construcción","Garajes","Sobre","Cuota inicial","Vivienda Social"
        ]

        salida = {"URL": url, "Precio": precio}
        for campo in campos:
            salida[campo] = data.get(campo, "No encontrado")

        return salida

    except Exception as e:
        return {"URL": url, "Error": str(e)}


In [31]:
# 🔁 Recorremos varias páginas
resultados = []
pagina = 1
paginas_vacias_consecutivas = 0
LIMITE_VACIAS = 10
LIMITE_TOTAL_PAGINAS = 300 # o más si deseas

try:
    while pagina <= LIMITE_TOTAL_PAGINAS:
        print(f"\n📄 Procesando página {pagina}")
        urls = obtener_urls_pagina(pagina)

        if not urls:
            paginas_vacias_consecutivas += 1
            print(f"⚠️ Página vacía #{paginas_vacias_consecutivas}")
            if paginas_vacias_consecutivas >= LIMITE_VACIAS:
                print(f"🛑 {LIMITE_VACIAS} páginas vacías consecutivas. Se detiene.")
                break
        else:
            paginas_vacias_consecutivas = 0
            for i, link in enumerate(urls, 1):
                print(f"  [{i}/{len(urls)}] {link}")
                resultados.append(scrap_propiedad(link))
                time.sleep(1)

        pagina += 1

except KeyboardInterrupt:
    print("🛑 Interrupción manual detectada. Guardando progreso...")

finally:
    df = pd.DataFrame(resultados)
    df.to_csv("viviendas_lima_paginas_departamentos.csv", index=False, encoding="utf-8-sig")
    print(f"✅ Se guardaron {len(resultados)} registros en 'viviendas_lima_paginas_departamentos.csv'")



📄 Procesando página 1
  [1/5] https://www.infocasas.com.pe/vendo-espectacular-penthouse-con-azotea-privada/192545274


C:\Users\arthu\AppData\Local\Temp\ipykernel_34724\1796746978.py:31: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  script_tags = s.find_all("script", text=re.compile("TechnicalSheet"))


  [2/5] https://www.infocasas.com.pe/vendo-exclusivo-de-estreno-departamento-en-miraflores-jardin-privado-y-ubicacion-perfecta/192014554
  [3/5] https://www.infocasas.com.pe/vendo-dpto-3-dorm-jesus-maria-4to-piso-calle-rio-de-janeiro-con-ascensor/192439323
  [4/5] https://www.infocasas.com.pe/vendo-departamento-duplex-piso-1-2-amplios-dormitorios-con-estacionamiento/192560928
  [5/5] https://www.infocasas.com.pe/vendo-departamento-en-la-molina-la-ensenada/191932499

📄 Procesando página 2
  [1/1] https://www.infocasas.com.pe/vendo-duplex-en-el-condominio-el-nuevo-rancho-de-miraflores/192232644

📄 Procesando página 3
  [1/3] https://www.infocasas.com.pe/vendo-departamento-en-los-tusilagos-sj-lurigancho/192108355
  [2/3] https://www.infocasas.com.pe/vendo-departamento-de-82-m2-con-balcon-con-vista-a-la-calle-en-lince/190525556
  [3/3] https://www.infocasas.com.pe/vendo-departamento-de-103-m2-en-28-julio-346-miraflores/192024213

📄 Procesando página 4
⚠️ Página vacía #1

📄 Procesando págin